In [2]:
### Import necessary modules
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

### Prepare synthetic data (replace with your actual dataset)
# Assuming 1000 samples, each image of size (224, 224, 3), and 10 classes
num_samples = 1000
num_classes = 10

# Generate dummy data for images
trainX = np.random.rand(num_samples, 224, 224, 3).astype('float32')  # Normalized pixel values
# Generate dummy labels
trainY = np.random.randint(0, num_classes, num_samples)
trainY = to_categorical(trainY, num_classes=num_classes)  # One-hot encode labels

# Optionally split into training and testing sets (if not using validation_split)
trainX, testX, trainY, testY = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

### Build a classifier based on a pre-trained model
# Load VGG16 with pretrained weights and without the top dense layers
vgg16_model = vgg16.VGG16(input_shape=(224, 224, 3), include_top=False)

# Build a new model based on pre-trained VGG16
inputs = vgg16_model.inputs
x = vgg16_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs, outputs, name='NewClassifier')
model.summary(show_trainable=True)

### Freeze weights of feature extractor and train only newly added head
for layer in model.layers[:-4]:
    layer.trainable = False
model.summary(show_trainable=True)

### Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train model (Transfer Learning Phase)
model.fit(trainX, trainY, validation_split=0.1, epochs=10, batch_size=32)  # Reduced epochs for faster training

### Unfreeze some layers of feature extractor and retrain model (Fine-Tuning Phase)
for layer in model.layers[-6:-4]:
    layer.trainable = True
model.summary(show_trainable=True)

# Retrain model
model.fit(trainX, trainY, validation_split=0.1, epochs=10, batch_size=32)

### Unfreeze more layers of feature extractor and retrain model (Further Fine-Tuning Phase)
for layer in model.layers[-9:-6]:
    layer.trainable = True
model.summary(show_trainable=True)

# Retrain model
model.fit(trainX, trainY, validation_split=0.1, epochs=10, batch_size=32)

Model: "NewClassifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer (InputLayer)    │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_conv1 (Conv2D)       │ (None, 224, 224, 64)  │      1,792 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_conv2 (Conv2D)       │ (None, 224, 224, 64)  │     36,928 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_pool (MaxPooling2D)  │ (None, 112, 112, 64)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_conv1 (Conv2D)       │ (None, 112, 112, 128) │     73,856 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_conv2 (Conv2D)       │ (None, 112, 112, 128) │    147,584 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_pool (MaxPooling2D)  │ (None, 56, 56, 128)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv1 (Conv2D)       │ (None, 56, 56, 256)   │    295,168 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv2 (Conv2D)       │ (None, 56, 56, 256)   │    590,080 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv3 (Conv2D)       │ (None, 56, 56, 256)   │    590,080 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_pool (MaxPooling2D)  │ (None, 28, 28, 256)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv1 (Conv2D)       │ (None, 28, 28, 512)   │  1,180,160 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv2 (Conv2D)       │ (None, 28, 28, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv3 (Conv2D)       │ (None, 28, 28, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_pool (MaxPooling2D)  │ (None, 14, 14, 512)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv1 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv2 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv3 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_pool (MaxPooling2D)  │ (None, 7, 7, 512)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ flatten (Flatten)           │ (None, 25088)         │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 256)           │  6,422,784 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 10)            │      2,570 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 21,140,042 (80.64 MB)

 Trainable params: 21,140,042 (80.64 MB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 33,990,752 (129.66 MB)

 Trainable params: 15,864,586 (60.52 MB)

 Non-trainable params: 5,275,456 (20.12 MB)

 Optimizer params: 12,850,710 (49.02 MB)

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.3360 - loss: 1.8958 - val_accuracy: 0.0875 - val_loss: 2.4798
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.4210 - loss: 1.8153 - val_accuracy: 0.1125 - val_loss: 2.5747
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.4058 - loss: 1.8207 - val_accuracy: 0.0500 - val_loss: 2.5817
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.5115 - loss: 1.7016 - val_accuracy: 0.1500 - val_loss: 2.5470
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.4844 - loss: 1.7097 - val_accuracy: 0.1125 - val_loss: 2.5621
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.5215 - loss: 1.6551 - val_accuracy: 0.1125 - val_loss: 2.5553
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.5135 - loss: 1.6282 - val_accuracy: 0.0750 - val_loss: 2.7269
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 77s 3s/step - accuracy: 0.4193 - loss: 1.6768 - val_accuracy: 0.1375 - val_loss: